In [1]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)


Mounted at /content/gdrive/


In [91]:
import pandas as pd
import json
import networkx as nx
import pickle
import requests


In [3]:
df=pd.read_csv('/content/gdrive/My Drive/IIAProject/Dataset/graph/tmdb_5000_credits.csv')

In [4]:
cast=df.cast.tolist()

In [5]:
crew=df.crew.tolist()

In [6]:
crew[0]

'[{"credit_id": "52fe48009251416c750aca23", "department": "Editing", "gender": 0, "id": 1721, "job": "Editor", "name": "Stephen E. Rivkin"}, {"credit_id": "539c47ecc3a36810e3001f87", "department": "Art", "gender": 2, "id": 496, "job": "Production Design", "name": "Rick Carter"}, {"credit_id": "54491c89c3a3680fb4001cf7", "department": "Sound", "gender": 0, "id": 900, "job": "Sound Designer", "name": "Christopher Boyes"}, {"credit_id": "54491cb70e0a267480001bd0", "department": "Sound", "gender": 0, "id": 900, "job": "Supervising Sound Editor", "name": "Christopher Boyes"}, {"credit_id": "539c4a4cc3a36810c9002101", "department": "Production", "gender": 1, "id": 1262, "job": "Casting", "name": "Mali Finn"}, {"credit_id": "5544ee3b925141499f0008fc", "department": "Sound", "gender": 2, "id": 1729, "job": "Original Music Composer", "name": "James Horner"}, {"credit_id": "52fe48009251416c750ac9c3", "department": "Directing", "gender": 2, "id": 2710, "job": "Director", "name": "James Cameron"},

In [7]:
df.columns

Index(['movie_id', 'title', 'cast', 'crew'], dtype='object')

In [8]:
movie=df.movie_id.tolist()
title=df.title.tolist()

In [9]:
actors=set()
actors2=[]

In [10]:
movieMap={}
for i in range(int(len(cast)/10)):
  d=cast[i]
  obj = json.loads(d)
  obj=obj[:3]
  actors=[]
  obj2 = json.loads(crew[i])
  crews=set()
  for c in obj2:
    if c['job']=='Casting' or c['job']=='Director' or c['job']=='Producer':
      crews.add((c['id'],c['name'],c['job']))
  for a in obj:
      crews.add((a['id'],a['name'],'actor'))
  movieMap[movie[i]]=list(crews)

In [11]:
lst=[]
for item in movieMap:
  lst+=movieMap[item]
lst=set(lst)

In [98]:
name='Tom Cruise'
api_url = 'https://api.api-ninjas.com/v1/celebrity?name={}'.format(name)
response = requests.get(api_url, headers={'X-Api-Key': 'I3HLYoEMA163se9Tu1LOZQ==2yaIidMHnjvzXREw'})
if response.status_code == requests.codes.ok:
    txt = json.loads(response.text)

In [99]:
txt

[{'name': 'tom cruise',
  'net_worth': 600000000,
  'gender': 'male',
  'nationality': 'us',
  'occupation': ['actor',
   'film_producer',
   'screenwriter',
   'television_director'],
  'height': 1.7,
  'birthday': '1962-07-03',
  'age': 60,
  'is_alive': True}]

In [113]:
g=nx.Graph()
for item in lst:
    name=item[1]
    api_url = 'https://api.api-ninjas.com/v1/celebrity?name={}'.format(name)
    response = requests.get(api_url, headers={'X-Api-Key': 'I3HLYoEMA163se9Tu1LOZQ==2yaIidMHnjvzXREw'})
    txt=None
    if response.status_code == requests.codes.ok:
        txt = json.loads(response.text)
    worth=''
    nationality=''
    gender=''
    if txt:
      for i in txt:
        f=0
        try:
          for occ in i['occupation']:
            if item[2].lower() in occ:
              f=1
              break
          if f==1:
            try:
              nationality =i["nationality"]
              worth=i["net_worth"]
              gender=i["gender"] 
            except:
              pass
            break
        except:
            pass
    g.add_node(item[0],name=item[1],job=item[2],nationality=nationality,worth=worth,gender=gender)

In [130]:
mapDict={}
for item in movieMap:
  sorted(movieMap[item],key=lambda x: x[1])
  for i in range(len(movieMap[item])-1):
    for j in range(i+1,len(movieMap[item])):
      if movieMap[item][i][0]!=movieMap[item][j][0]:
        if (movieMap[item][i],movieMap[item][j]) not in mapDict.keys():
          mapDict[(movieMap[item][i],movieMap[item][j])]=1
        else:
          mapDict[(movieMap[item][i],movieMap[item][j])]+=1

In [131]:
len(mapDict),len(lst)

(14293, 1939)

In [15]:
!pip install scipy==1.8.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 41.6 MB 1.5 MB/s 
  Attempting uninstall: scipy
    Found existing installation: scipy 1.7.3
    Uninstalling scipy-1.7.3:
      Successfully uninstalled scipy-1.7.3


In [132]:
for pair in mapDict:
  g.add_edge(pair[0][0],pair[1][0], weight=mapDict[pair])

In [ ]:
with open('graph.pkl', 'wb') as f:
    pickle.dump(g, f)

In [133]:
ga=g.subgraph([i for i in list(g.nodes) if g.nodes[i]['job']=='actor'])
gp=g.subgraph([i for i in list(g.nodes) if g.nodes[i]['job']=='Producer'])
gd=g.subgraph([i for i in list(g.nodes) if g.nodes[i]['job']=='Director'])
gc=g.subgraph([i for i in list(g.nodes) if g.nodes[i]['job']=='Casting'])

In [134]:
print("Number of Actor Nodes-",ga.number_of_nodes())
print("Number of Producer Nodes-",gp.number_of_nodes())
print("Number of Director Nodes-",gd.number_of_nodes())
print("Number of Casting Director Nodes-",gc.number_of_nodes())
print("Total Number of Nodes-",g.number_of_nodes())

Number of Actor Nodes- 716
Number of Producer Nodes- 622
Number of Director Nodes- 246
Number of Casting Director Nodes- 241
Total Number of Nodes- 1825


In [135]:
print("Density of Graph",nx.density(g))

Density of Graph 0.00785267964431627


In [136]:
deg=list(g.degree)
dega=[(g.nodes[i[0]]['name'],i[1]) for i in deg if g.nodes[i[0]]['job']=='actor']
degp=[(g.nodes[i[0]]['name'],i[1]) for i in deg if g.nodes[i[0]]['job']=='Producer']
degd=[(g.nodes[i[0]]['name'],i[1]) for i in deg if g.nodes[i[0]]['job']=='Director']
degc=[(g.nodes[i[0]]['name'],i[1]) for i in deg if g.nodes[i[0]]['job']=='Casting']

In [137]:
dega.sort(key=lambda x: x[1],reverse=True)
degp.sort(key=lambda x: x[1],reverse=True)
degd.sort(key=lambda x: x[1],reverse=True)
degc.sort(key=lambda x: x[1],reverse=True)

In [138]:
print('Actors')
dega[:5]

Actors


[('Angelina Jolie', 86),
 ('Tom Cruise', 79),
 ('Leonardo DiCaprio', 77),
 ('Brad Pitt', 75),
 ('Arnold Schwarzenegger', 74)]

In [141]:
print('Directors')
degd[:5]

Directors


[('Ridley Scott', 48),
 ('Michael Bay', 48),
 ('Ron Howard', 41),
 ('Peter Jackson', 37),
 ('Don Hahn', 31)]

In [142]:
print('Prodcuers')
degp[:5]

Prodcuers


[('Neal H. Moritz', 70),
 ('Hugh Jackman', 69),
 ('Tom Hanks', 65),
 ('Jerry Bruckheimer', 63),
 ('Larry J. Franco', 63)]

In [36]:
print('Casting Directors')
degc[:5]

Casting Directors


[('Lucinda Syson', 113),
 ('Sarah Finn', 112),
 ('John Papsidera', 103),
 ('Ronna Kress', 93),
 ('Mary Hidalgo', 93)]

In [42]:
cl=nx.closeness_centrality(g)
cl=list(cl.items())
cla=[(g.nodes[i[0]]['name'],i[1]) for i in cl if g.nodes[i[0]]['job']=='actor']
clp=[(g.nodes[i[0]]['name'],i[1]) for i in cl if g.nodes[i[0]]['job']=='Producer']
cld=[(g.nodes[i[0]]['name'],i[1]) for i in cl if g.nodes[i[0]]['job']=='Director']
clc=[(g.nodes[i[0]]['name'],i[1]) for i in cl if g.nodes[i[0]]['job']=='Casting']

In [47]:
cla.sort(key=lambda x: x[1],reverse=True)
clp.sort(key=lambda x: x[1],reverse=True)
cld.sort(key=lambda x: x[1],reverse=True)
clc.sort(key=lambda x: x[1],reverse=True)

In [48]:
print('Actors')
cla[:5]

Actors


[('Angelina Jolie', 0.39221533554025845),
 ('Brad Pitt', 0.3842750473235369),
 ('Tom Cruise', 0.3796299643339117),
 ('Will Smith', 0.37607584100137126),
 ('Johnny Depp', 0.3737970867170089)]

In [46]:
print('Directors')
cld[:5]

Directors


[('Brad Peyton', 0.29745416526938145),
 ('Mike Newell', 0.2994653844867022),
 ('Jonathan Mostow', 0.32022920610294736),
 ('Peter Hyams', 0.26367216268039967),
 ('Will Finn', 0.2581937724543047)]

In [53]:
print('Casting Directors')
clc[:5]

Casting Directors


[('Lucinda Syson', 0.40386166418501246),
 ('Sarah Finn', 0.39248269432385785),
 ('Ronna Kress', 0.38651070434533413),
 ('John Papsidera', 0.3841041444783852),
 ('Janet Hirshenson', 0.3834220505481239)]

In [54]:
print('Prodcuers')
clp[:5]

Prodcuers


[('Brian Grazer', 0.3778858756769412),
 ('Neal H. Moritz', 0.3742830634278003),
 ('Jerry Bruckheimer', 0.3735545713060766),
 ('Tom Hanks', 0.3690846875468586),
 ('Hugh Jackman', 0.36595685121171573)]

In [62]:
bl=nx.betweenness_centrality(g)
bl=list(bl.items())
bla=[(g.nodes[i[0]]['name'],i[1]) for i in bl if g.nodes[i[0]]['job']=='actor']
blp=[(g.nodes[i[0]]['name'],i[1]) for i in bl if g.nodes[i[0]]['job']=='Producer']
bld=[(g.nodes[i[0]]['name'],i[1]) for i in bl if g.nodes[i[0]]['job']=='Director']
blc=[(g.nodes[i[0]]['name'],i[1]) for i in bl if g.nodes[i[0]]['job']=='Casting']

In [63]:
bla.sort(key=lambda x: x[1],reverse=True)
blp.sort(key=lambda x: x[1],reverse=True)
bld.sort(key=lambda x: x[1],reverse=True)
blc.sort(key=lambda x: x[1],reverse=True)

In [64]:
print('Actors')
bla[:5]

Actors


[('Arnold Schwarzenegger', 0.028147915880146313),
 ('Angelina Jolie', 0.026644584336829428),
 ('Tom Cruise', 0.022563294670007693),
 ('Brad Pitt', 0.021984697846443822),
 ('Leonardo DiCaprio', 0.01987926007647464)]

In [65]:
print('Prodcuers')
blp[:5]

Prodcuers


[('Tom Hanks', 0.03110445522495553),
 ('Neal H. Moritz', 0.01725756220890225),
 ('Brian Grazer', 0.017115182303862258),
 ('Hugh Jackman', 0.017077584236605502),
 ('Douglas Wick', 0.01679204471971285)]

In [66]:
print('Directors')
bld[:5]

Directors


[('Ridley Scott', 0.011246156997316846),
 ('Don Hahn', 0.00965904006635408),
 ('Michael Bay', 0.008490881118711116),
 ('Brad Bird', 0.0065126696322548566),
 ('Paul Verhoeven', 0.005227935012895766)]

In [119]:
print('Casting Directors')
blc[:5]

Casting Directors


[('Mary Hidalgo', 0.05008523600644913),
 ('Lucinda Syson', 0.04087103030419087),
 ('Sarah Finn', 0.0358973222450102),
 ('John Papsidera', 0.03557092459283543),
 ('Avy Kaufman', 0.03149010845361335)]

In [84]:
al=nx.clustering(g)
al=list(al.items())
ala=[(g.nodes[i[0]]['name'],i[1]) for i in al if g.nodes[i[0]]['job']=='actor' and int(i[1])!=1]
alp=[(g.nodes[i[0]]['name'],i[1]) for i in al if g.nodes[i[0]]['job']=='Producer'  and int(i[1])!=1]
ald=[(g.nodes[i[0]]['name'],i[1]) for i in al if g.nodes[i[0]]['job']=='Director'  and int(i[1])!=1]
alc=[(g.nodes[i[0]]['name'],i[1]) for i in al if g.nodes[i[0]]['job']=='Casting'  and int(i[1])!=1]

In [86]:
ala.sort(key=lambda x: x[1],reverse=True)
alp.sort(key=lambda x: x[1],reverse=True)
ald.sort(key=lambda x: x[1],reverse=True)
alc.sort(key=lambda x: x[1],reverse=True)

In [87]:
print('Actors')
ala[:5]

Actors


[('Tom Hiddleston', 0.9642857142857143),
 ('Josh Duhamel', 0.9454545454545454),
 ('Theo James', 0.9090909090909091),
 ('Shailene Woodley', 0.9090909090909091),
 ('Gwyneth Paltrow', 0.8888888888888888)]

In [88]:
print('Casting Directors')
alc[:5]

Casting Directors


[('Robin Gurland', 0.9333333333333333),
 ('Marianne Stanicheva', 0.9272727272727272),
 ('Venus Kanani', 0.9090909090909091),
 ('Jennifer Rudnicke', 0.8241758241758241),
 ('Avi Lerner', 0.7948717948717948)]

In [89]:
print('Prodcuers')
alp[:5]

Prodcuers


[('Nina Jacobson', 0.9523809523809523),
 ('Bill Miller', 0.9523809523809523),
 ('Susan Downey', 0.9454545454545454),
 ('Dan Lin', 0.9454545454545454),
 ('Jerry Weintraub', 0.9333333333333333)]

In [90]:
print('Directors')
ald[:5]

Directors


[('Guy Ritchie', 0.9454545454545454),
 ('Steven Soderbergh', 0.9333333333333333),
 ('Joe Russo', 0.9166666666666666),
 ('Anthony Russo', 0.9166666666666666),
 ('Joss Whedon', 0.8928571428571429)]